# TorchSpatial Tutorial


# Data Download

In [ ]:
# Set the corresponding data path:
/data2/nw33659/TorchSpatial/main/paths.py

## Location Encoder Usage Example

In [ ]:
import torch
import numpy as np

from space2vec.SpatialRelationEncoder import *
from space2vec.module import *
from space2vec.data_utils import *
from space2vec.utils import *

IPython will make a temporary file named: /var/folders/y5/4m7d403558sgz0sldvc7m5w40000gn/T/ipython_edit_fycjtau9/ipython_edit_q2e4ayc8.py


In [ ]:
# specify the params for the location encoder, current spa_embed_dim is 128
params = {
    'spa_enc_type': 'gridcell',
    'spa_embed_dim': 128, 
    'extent': (0, 200, 0, 200),
    'freq': 16,
    'max_radius': 283,
    'min_radius': 1,
    'spa_f_act': "leakyrelu",
    'freq_init': 'geometric',
    'num_hidden_layer': 1,
    'dropout': 0.5,
    'hidden_dim': 512,
    'use_layn': True,
    'skip_connection': True,
    'num_rbf_anchor_pts': 0,
    'rbf_kernal_size': 0,
    'spa_enc_use_postmat': True,
    'device': 'cpu'
}

In [ ]:
loc_enc = get_spa_encoder(
                train_locs = [],
                params = params, 
                spa_enc_type = params['spa_enc_type'], 
                spa_embed_dim = params['spa_embed_dim'], 
                extent = params['extent'], 
                coord_dim = 2,
                frequency_num = params['freq'], 
                max_radius = params['max_radius'], 
                min_radius = params['min_radius'], 
                f_act = params['spa_f_act'], 
                freq_init = params['freq_init'],
                num_rbf_anchor_pts = params['num_rbf_anchor_pts'], 
                rbf_kernal_size = params['rbf_kernal_size'],  
                use_postmat = params['spa_enc_use_postmat'], 
                device = params['device']).to(params['device'])

/Users/kuratome/Downloads/location_encoder/space2vec/module.py:96: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.linear.weight)


In [ ]:
# synthetic coords data
batch_size, coord_dim = 10, 2
coords = np.random.randint(1, 201, size=(batch_size, coord_dim))

# coords: shape [batch_size, 2]
coords 

array([[ 55,  71],
       [115, 187],
       [188,  87],
       [176, 143],
       [157,  10],
       [ 88,  40],
       [ 88, 189],
       [175,  87],
       [182,  61],
       [100, 104]])

In [ ]:
coords = np.array(coords)
coords = np.expand_dims(coords, axis=1)
loc_embeds = torch.squeeze(loc_enc(coords))

# loc_embed: shape [batch_size, spa_embed_dim]
loc_embeds 

tensor([[-0.1479,  1.3423, -0.0000,  ...,  0.4105,  1.9410,  0.5589],
        [-0.0000,  1.2338,  3.6367,  ...,  0.0000,  0.0000,  3.0991],
        [-0.7992,  2.9915, -0.0000,  ..., -0.0000,  0.0000, -0.0000],
        ...,
        [ 0.1175,  0.0000,  0.0000,  ...,  0.0000, -0.4429, -0.0000],
        [ 0.0000, -0.0880, -0.2827,  ...,  0.0000, -0.5837, -0.8116],
        [-0.0000,  0.0000, -0.0000,  ..., -0.1569, -0.3970, -0.7252]],
       grad_fn=<SqueezeBackward0>)

In [ ]:
# Initialize the encoder
encoder = GridLookupSpatialRelationLocationEncoder(
    spa_embed_dim=64,
    extent=(-180, 180, -90, 90),  # Example extent
    interval=1000000,  # Example interval
    coord_dim=2,
    device="cuda",
    ffn_act="relu",
    ffn_num_hidden_layers=1,
    ffn_dropout_rate=0.5,
    ffn_hidden_dim=256,
    ffn_use_layernormalize=True,
    ffn_skip_connection=True,
    ffn_context_str="GridLookupSpatialRelationEncoder"
)

coords = np.array([[34.0522, -118.2437], [40.7128, -74.0060]])  # Example coordinate data
embeddings = encoder.forward(coords)

In [ ]:
!wget https://figshare.com/ndownloader/files/47020978


In [ ]:
!tar -xvf 47020978

In [ ]:
!python3 main/main.py \
    --save_results T \
    --load_super_model T \
    --num_epochs 0 \
    --spa_enc_type NeRF \
    --meta_type ebird_meta \
    --dataset birdsnap \
    --eval_split test \
    --frequency_num 64 \
    --min_radius 0.02 \
    --max_radius 1 \
    --num_hidden_layer 1 \
    --hidden_dim 256 \
    --spa_f_act leakyrelu \
    --unsuper_lr 0.1 \
    --lr 0.01 \
    --model_dir ../models/nerf/ \
    --train_sample_ratio 1.0 \
    --device cuda:0


In [ ]:
!pip install matplotlib